In [71]:
import pandas as pd
import re
import csv
import polars as pl

In [72]:
def payer_category(x):
    payers = {
        'Gross Charge': 'gross',
        'Discounted Cash Price': 'cash',
        'Minimum Negotiated Charge': 'min',
        'Maximum Negotiated Charge': 'max'
    }

    try:
        return payers[x]
    except KeyError:
        return 'payer'

In [73]:
file = 'pricing-transparency-charge.csv'
df = pd.read_csv(file)

In [74]:
cols = df.columns.tolist()
drop_cols = [col for col in cols if 'Unnamed' in col]
df.drop(drop_cols, axis=1, inplace=True)
print(df.columns.tolist())

['CDM Item Number', 'Revenue Code', 'Service ID', 'Service Mod1', 'Service Mod2', 'Service Description', ' Gross Charge ', ' Discounted Cash Price ', ' Minimum Negotiated Charge ', ' Maximum Negotiated Charge ', ' Payer Negotiated Charge: AARP (Plan: All) ', ' Payer Negotiated Charge: AARP- MedicareComplete UnitedHealthCare (Plan: All) ', ' Payer Negotiated Charge: Adminastar Blue Cross of Kentucky (Plan: All) ', ' Payer Negotiated Charge: Aetna (Plan: All) ', ' Payer Negotiated Charge: Aetna Better Health (LA) (Plan: All) ', ' Payer Negotiated Charge: Aetna Medicare Advantage (Plan: All) ', ' Payer Negotiated Charge: Blue Advantage of LA (Plan: All) ', ' Payer Negotiated Charge: Blue Cross Blue Shield of LA (Plan: All) ', ' Payer Negotiated Charge: Blue Cross Blue Shield of LA MCR Adv (Plan: All) ', ' Payer Negotiated Charge: Blue Cross Blue Shield of TX (Plan: All) ', ' Payer Negotiated Charge: CHC Louisiana (Plan: All) ', ' Payer Negotiated Charge: Cigna (Plan: All) ', ' Payer Negot

In [75]:
df.rename(columns={
    'CDM Item Number': 'local_code',
    'Revenue Code': 'rev_code',
    'Service ID': 'code',
    'Service Description': 'description',

}, inplace=True)

In [76]:
cols = df.columns.tolist()
id_vars = cols[:6]
value_vars = cols[6:]
df = pd.melt(df, id_vars=id_vars, value_vars=value_vars, var_name='payer', value_name='standard_charge')

In [77]:
df['modifiers'] = df['Service Mod1'] + "|" + df['Service Mod2']
df.drop(columns=['Service Mod1', 'Service Mod2'], inplace=True)

In [78]:
df['payer'] = df['payer'].str.strip()

In [79]:
df['hcpcs_cpt'] = df['code'].str[:5]
df.loc[df['code'].str.len() == 7, 'modifiers'] = df['code'].str[-2:]

In [80]:
df['payer_category'] = df['payer'].apply(payer_category)

In [81]:
df['hospital_id'] = '190118'
df['rev_code'] = df['rev_code'].astype(str)
df['rev_code'] = df['rev_code'].str.zfill(4)

df['standard_charge'] = df['standard_charge'].str.replace('$', '').str.replace(',', '')
df['standard_charge'] = df['standard_charge'].str.strip()
df['code'] = df['code'].fillna('')
df['hcpcs_cpt'] = df['hcpcs_cpt'].fillna('')
df['modifiers'].fillna('', inplace=True)


C:\Users\adria\AppData\Local\Temp\ipykernel_4744\3848457064.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['standard_charge'] = df['standard_charge'].str.replace('$', '').str.replace(',', '')


In [82]:
df

,local_code,rev_code,code,description,payer,standard_charge,modifiers,hcpcs_cpt,payer_category,hospital_id
0,100000,0761,Q3014,TELEHEALTH ORIGINATING SITE FEE,Gross Charge,41.00,,Q3014,gross,190118
1,1000000,0300,86850,BB ANTIBODY SCREEN,Gross Charge,181.00,,86850,gross,190118
2,1000001,0300,99050,LBC-LAB FEE NON ROUTINE,Gross Charge,90.00,,99050,gross,190118
3,1000002,0300,99001,COURIER FEE-LBC/LESS THAN 60 MILES,Gross Charge,113.00,,99001,gross,190118
4,1000003,0300,86235,ANTI-SSA AND SSB PANEL,Gross Charge,324.00,,86235,gross,190118
...,...,...,...,...,...,...,...,...,...,...
199495,9500011,0942,97802,MNT IND INIT 15 MINS,Payer Negotiated Charge: WellCare of Louisiana...,NaN,,97802,payer,190118
199496,9500012,0942,97803,MNT IND. REASSESSMNT 15 MINS,Payer Negotiated Charge: WellCare of Louisiana...,NaN,,97803,payer,190118
199497,9500013,0942,97804,MNT GRP 30 MINS,Payer Negotiated Charge: WellCare of Louisiana...,NaN,,97804,payer,190118
199498,9500014,0942,G0270,MNT 2ND DX INIT,Payer Negotiated Charge: WellCare of Louisiana...,NaN,,G0270,payer,190118


In [83]:
df.dropna(subset=['standard_charge'], inplace=True)
df = df[pd.to_numeric(df['standard_charge'], errors='coerce').notnull()]

In [84]:
df1 = pl.DataFrame(df)
df1.write_csv('test.csv')
# df.to_csv('desoto.csv', index=False)
